## Content Creation with a multi-agent approach.

In [ ]:
import logging
import json
import textwrap
import re

from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [ ]:
# This just helps to make the output more readable.
# Just click play on the cell to run it, and move on. :)

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class Article(BaseModel):
    title: str
    content: str
    call_to_action: str

class ContentOutput(BaseModel):
    article: Article
    social_media_posts: List[SocialMediaPost]

In [ ]:
# Some casual things for formatting.

intermediate_steps: list[ChatMessageContent] = []

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)

output_settings = AzureChatPromptExecutionSettings(response_format=ContentOutput)

In [ ]:
# Create and configure the kernel, along with the plugins. This is what makes it all work.

# The Search plugin is used to search for information on the web. This allows us to get more up-to-date information.
from search import SearchPlugin

service_id: str = "default_service_id"
kernel = Kernel(service_id=service_id)

kernel.add_plugin(SearchPlugin(), plugin_name="search")
kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4o"))

In [ ]:
# Create the Lead Market Analyst agent. NOTE: The name of the agent must not contain spaces.
lead_research_agent = ChatCompletionAgent(
    service= AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="LeadResearchAgent",
    instructions=(
    """
    As the Lead Research Analyst at a premier content platform, you specialize in finding information that best fits the needs of your
    audience. Your primary responsibility is to ensure that the content produced by your team is not only accurate but also timely and
    relevant. You will do this by conducting in-depth research. You will need to search the web to get this content.
    """),
    plugins=[
        SearchPlugin()
    ],
    
)

In [ ]:
# Create the Data Analyst agent. NOTE: The name of the agent must not contain spaces.
data_analyst_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="DataAnalystAgent",
    instructions=(
    """
    As the Chief Data Strategist at a leading advisory firm, your expertise lies in analyzing vast datasets to uncover trends and
    opportunities that inform high-impact strategies with the goal of Synthesizing complex data into actionable insights 
    that can be transformed into compelling content.
    """),
)

In [ ]:
# Create the Content Creator agent. NOTE: The name of the agent must not contain spaces.
blog_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="BlogCreatorAgent",
    instructions=(
    """
    As the Content Creator at a leading advisory firm, your role is to transform complex data and insights into
    engaging and informative content that resonates with our audience. You will be responsible for creating articles, blogs, and other
    materials that effectively communicate our findings and recommendations. Additionally, you will collaborate with other agents to ensure
    the content aligns with our overall strategy and objectives.
    """),
)

In [ ]:
# Create the Social Media Manager agent. NOTE: The name of the agent must not contain spaces.
social_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="SocialCreatorAgent",
    instructions=(
    """
    As the Social Media Manager at a leading advisory firm, your role is to create engaging and impactful content for our social media
    platforms. You will be responsible for crafting tweets, posts, and other materials that effectively communicate our findings and
    recommendations. Any twitter posts, Instagram posts, and other posts should be very short and easy to understand, and should sound like clickbait.
    Additionally, you will collaborate with other agents to ensure the content aligns with our overall strategy and objectives.
    """),
)

In [ ]:
# Create the Quality Assurance agent. NOTE: The name of the agent must not contain spaces.
quality_assurance_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="QualityAssuranceAgent",
    arguments=KernelArguments(settings=output_settings),
    instructions=(
    """
    As the Quality Assurance Specialist at a leading advisory firm, your role is to ensure that all content produced meets our
    high standards for accuracy, clarity, and relevance. You will review and edit content created by the team to ensure it aligns with our
    brand voice and effectively communicates our insights. Additionally, you will provide feedback to the social_creator_agent and the blog_creator_agent to enhance content quality.
    If the content is not good enough, you will need to explain why it is not good enough and what needs to be changed.
    If it is good enough, you will need to say that it is good enough and that it can be published.
    """),
)

In [ ]:
# The selection function is used to determine which agent should take the next turn in the conversation,
# based on the conversation history and the rules defined in the prompt.

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on the conversation history, determine the next agent to take a turn. 
    Only return the name of the agent from the following list:
    - LeadResearchAgent
    - DataAnalystAgent
    - BlogCreatorAgent
    - SocialCreatorAgent
    - QualityAssuranceAgent

    Rules:
    - LeadResearchAgent starts first.
    - After LeadResearchAgent, DataAnalystAgent speaks.
    - After DataAnalystAgent, BlogCreatorAgent speaks.
    - After BlogCreatorAgent speaks, the QualityAssuranceAgent speaks.
    - If QualityAssuranceAgent requests changes, restart the cycle with BlogCreatorAgent.
    - If the QualityAssuranceAgent approves the content, the SocialCreatorAgent speaks.
    - After SocialCreatorAgent speaks, the QualityAssuranceAgent speaks.
    - If the QualityAssuranceAgent requests changes, restart the cycle with SocialCreatorAgent.
    - If the QualityAssuranceAgent approves the content, the cycle ends.

    History:
    {{$history}}
    """,
)

In [ ]:
# The termination function is used to determine when the conversation should end.
# It checks if the content produced by the agents is complete and valid JSON.

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the structured content output (article and social_media_posts) is complete and valid JSON.
    If the output is complete and valid, respond with "yes". Otherwise, respond with "no".
    Ensure the JSON adheres to the schema provided.

    History:
    {{$history}}
    """
)

In [ ]:
# This creates the agent group that will manage the conversation between the agents.

agent_group = AgentGroupChat(
    agents=[lead_research_agent, data_analyst_agent, blog_creator_agent, social_creator_agent, quality_assurance_agent],
    termination_strategy = KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: any("yes" in item.content.lower() for item in result.value) if isinstance(result.value, list) else "yes" in result.value.content.lower(),
        history_variable_name="history",
        maximum_iterations=10
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: result.value[0].content.strip() if isinstance(result.value, list) and result.value else result.value.content.strip(),
        agent_variable_name="agents",
        history_variable_name="history"
    )
)

In [ ]:
# This is the question that will be asked to the agents. It should be a clear and concise question that the agents can work on together.
topic = "Inflation in the US and the impact on the stock market in 2025"


initial_message = f"Subject: {topic}\n\nCollaborate to produce structured content output matching the provided schema."
await agent_group.add_chat_message(message=initial_message)

final_response = ""
async for response in agent_group.invoke():
    print(f"# {response.name}: {response.content}\n")
    final_response = response.content

print("Complete.")

In [ ]:
# We run this to extract the JSON from the final response, and display the article and social media posts in a formatted way.

def extract_json(text: str) -> dict:
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return json.loads(match.group(1))
    return json.loads(text.strip())

output_data = extract_json(final_response)
article = output_data.get("article", {})
posts = output_data.get("social_media_posts", [])

article_md = f"# {article.get('title')}\n\n{article.get('content')}\n\n**{article.get('call_to_action')}**"
display(Markdown(article_md))

for post in posts:
    print(f"Platform: {post['platform']}")
    print(textwrap.fill(post['content'], width=60))
    print('-' * 60)